https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
    self.conv2 = nn.Sequential(
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
    self.downsample = downsample
    self.relu = nn.ReLU()
    self.out_channels = out_channels
  
  def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = self.conv2(out)
    if self.downsample:
      residual = self.downsample(x)
    out += residual
    out = self.relu(out)
    return out

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes = 10):
    super(ResNet, self).__init__()
    self.inplanes = 64
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
    self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
    self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
    self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
    self.avgpool = nn.AvgPool2d(7, stride=1)
    self.fc = nn.Linear(512, num_classes)

  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None
      if stride != 1 or self.inplanes != planes:
          
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
              nn.BatchNorm2d(planes),
          )
      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      self.inplanes = planes
      for i in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

## 1. 初始卷积层
在残差块之前，ResNet 通常开始于一个单独的卷积层，这个层用于初步提取特征：

卷积：使用较大的卷积核（如7x7或者较大的3x3），较大的步长（通常是2），和适当的填充来保持空间尺寸。这一步是为了从原始图像中提取粗略的特征，并减少特征图的尺寸。
批量归一化（Batch Normalization）：归一化处理帮助加快收敛速度，同时使模型更加稳定。
ReLU激活函数：为了引入非线性，使得网络能够学习更复杂的模式。
最大池化：可选的最大池化步骤进一步减小空间尺寸，这在处理非常大的图像时非常有用。
## 2. 残差块
残差块是ResNet的核心，每个块包括以下几个组件：

主路径：包含两个或三个卷积层，每个卷积层后面通常跟着批量归一化和ReLU激活函数。这些卷积层通常使用较小的3x3或1x1卷积核。
跳跃连接（Skip Connection）：将输入直接加到块的输出上。如果输入和输出的维度不匹配，可能需要通过一个1x1的卷积来调整通道数或使用步长来改变空间维度。
激活函数：在将跳跃连接的结果与主路径输出相加后，通常会应用一个非线性激活函数（如ReLU）。

## 3. 层间连接
每一组残差块后面通常有一个下采样步骤，用于减少特征图的空间尺寸并增加通道数，从而让网络逐渐学习更高级的特征表示。
这通常通过在残差块的第一个卷积层中设置较大的步长（如2），或者在跳跃连接中使用1x1卷积层实现。

## 4.结尾层
自适应平均池化（Adaptive Average Pooling）：在网络的最后，通常使用自适应平均池化层将特征图的空间尺寸降为1x1，这样每个通道只有一个数值，总结了整个特征图的信息。
全连接层：将池化后的特征图展平并通过一个或多个全连接层来进行最终的分类或其他任务。
输出：最后的输出层根据任务的不同，可能是一个分类层（使用softmax激活函数）或者其他类型的输出。